<a href="https://colab.research.google.com/github/Rayadi-Ayoub/Syst-me-intelligent-/blob/main/syst_me_intelligent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install kaggle

In [ ]:
files.upload()

NameError: name 'files' is not defined

In [ ]:
# Installe le package 'kaggle' depuis PyPI
# Ce package permet d'interagir avec l'API Kaggle (téléchargement de datasets, participation à des compétitions, etc.)
!pip install kaggle

# Importe le module 'files' de Google Colab
# Ce module permet d'importer ou exporter des fichiers entre ton ordinateur et l'environnement Colab
from google.colab import files

# Ouvre une boîte de dialogue pour importer le fichier kaggle.json
# Ce fichier contient tes identifiants d'API Kaggle (username et key)
files.upload()

# Crée un dossier caché nommé '.kaggle' dans le répertoire personnel (~)
# Ce dossier sert à stocker le fichier de configuration d'accès à l'API Kaggle
!mkdir -p ~/.kaggle

# Déplace le fichier 'kaggle.json' téléchargé vers le dossier '.kaggle'
!mv kaggle.json ~/.kaggle/

# Change les permissions du fichier 'kaggle.json'
# Le chiffre 600 signifie : seul le propriétaire du fichier peut le lire et l'écrire
# (c’est une mesure de sécurité exigée par l’API Kaggle)
!chmod 600 ~/.kaggle/kaggle.json



In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("simhadrisadaram/mimic-cxr-dataset")

print("Path to dataset files:", path)

In [ ]:
# Définit le chemin de base (base_path) vers le dossier local où est stocké le dataset téléchargé depuis Kaggle.
# Ici, le chemin pointe vers le cache interne de KaggleHub, utilisé par Colab pour stocker les datasets Kaggle.
# Le dossier correspond au dataset "mimic-cxr-dataset" de l’utilisateur "simhadrisadaram", version 2.
base_path = "/root/.cache/kagglehub/datasets/simhadrisadaram/mimic-cxr-dataset/versions/2"


In [ ]:
import os
import pandas as pd

base_path = "/root/.cache/kagglehub/datasets/simhadrisadaram/mimic-cxr-dataset/versions/2"

# Vérifie l'existence des fichiers
print("Contenu du dossier :", os.listdir(base_path))

train_csv = os.path.join(base_path, "mimic_cxr_aug_train.csv")
val_csv = os.path.join(base_path, "mimic_cxr_aug_validate.csv")

# Vérifie si les fichiers existent avant lecture
if not os.path.exists(train_csv):
    raise FileNotFoundError(f"Fichier d'entraînement introuvable : {train_csv}")
if not os.path.exists(val_csv):
    raise FileNotFoundError(f"Fichier de validation introuvable : {val_csv}")

df_train = pd.read_csv(train_csv)
df_val = pd.read_csv(val_csv)

print("Train shape:", df_train.shape)
print("Validation shape:", df_val.shape)
df_train.head()


In [ ]:
import ast  # pour convertir la chaîne en liste Python

def extract_image_path(path_str):
    try:
        path_list = ast.literal_eval(path_str)
        if isinstance(path_list, list) and len(path_list) > 0:
            return path_list[0]  # on garde le premier chemin
        else:
            return None
    except:
        return None

df_train['image_path'] = df_train['image'].apply(extract_image_path)
df_val['image_path'] = df_val['image'].apply(extract_image_path)

print(df_train[['subject_id', 'image_path', 'view']].head())


In [ ]:
# Importe le module 'ast' (Abstract Syntax Trees)
# Ce module permet de convertir une chaîne de caractères représentant une structure Python (ex: liste, dict)
# en un véritable objet Python de manière sécurisée (contrairement à eval()).
import ast

# Définition d'une fonction pour vérifier si une image a une vue frontale (PA ou AP)
def has_frontal_view(views_str):
    try:
        # Convertit la chaîne 'views_str' (par ex. "['PA', 'LATERAL']") en une vraie liste Python
        views = ast.literal_eval(views_str)

        # Vérifie si au moins un des éléments de la liste est 'PA' (Posteroanterior) ou 'AP' (Anteroposterior)
        # Ces deux types correspondent à des vues frontales de radiographies thoraciques
        return any(v in ['PA', 'AP'] for v in views)

    except:
        # En cas d'erreur (par exemple si la chaîne est vide ou mal formée), renvoie False
        return False

# Filtre le DataFrame d'entraînement pour ne garder que les lignes ayant une vue frontale
# La méthode .apply() applique la fonction 'has_frontal_view' à chaque valeur de la colonne 'view'
df_frontal = df_train[df_train['view'].apply(has_frontal_view)].copy()

# Affiche le nombre d’images frontales par rapport au total du dataset d'entraînement
print("Images frontales :", len(df_frontal), "/", len(df_train))

# Affiche les 3 premières lignes du DataFrame filtré pour vérification
df_frontal.head(3)


In [ ]:
# Importation du module 'os' : permet d’interagir avec le système d’exploitation
# (ex : naviguer dans les dossiers, construire des chemins de fichiers, lister des répertoires, etc.)
import os

# Importation d'OpenCV (cv2) : une bibliothèque très utilisée pour le traitement d'images et la vision par ordinateur
# Elle permet de lire, redimensionner, convertir et transformer les images.
import cv2

# Importation de NumPy : utilisée pour manipuler efficacement des tableaux numériques multidimensionnels
# (par ex. les matrices représentant des images)
import numpy as np

# Importation de Matplotlib : utilisée pour la visualisation (affichage d'images, tracés, histogrammes, etc.)
import matplotlib.pyplot as plt

# Importation de tqdm : affiche une barre de progression lors des boucles, très pratique pour suivre l’avancement du traitement de nombreuses images
from tqdm import tqdm

# Importation de TensorFlow : framework de deep learning pour construire, entraîner et exécuter des modèles neuronaux
import tensorflow as tf

# Importation de la fonction 'img_to_array' de Keras (module inclus dans TensorFlow)
# Elle permet de convertir une image PIL (ou un tableau OpenCV) en tableau NumPy adapté aux modèles de deep learning
from tensorflow.keras.utils import img_to_array


In [ ]:
# Définit la taille standard à laquelle les images seront redimensionnées
# (224x224 pixels) — c’est une taille courante pour les modèles de deep learning
# tels que ResNet, VGG, DenseNet, etc.
IMG_SIZE = (224, 224)

# Définit le chemin de base où sont stockées les images du dataset MIMIC-CXR
# Ce répertoire contient les images officielles utilisées pour l’entraînement et la validation
base_images = "/root/.cache/kagglehub/datasets/simhadrisadaram/mimic-cxr-dataset/versions/2/official_data_iccv_final"

# Définition d'une fonction utilitaire pour obtenir le chemin absolu complet d'une image
def full_image_path(rel_path):
    # Remplace "files/" par une chaîne vide si présent dans le chemin relatif
    # Cela permet d’éviter les doublons dans le chemin (par ex. '.../official_data_iccv_final/files/...').
    # Ensuite, on assemble ce chemin corrigé avec 'base_images' grâce à os.path.join()
    return os.path.join(base_images, rel_path.replace("files/", ""))



In [ ]:
# Fonction de chargement et de prétraitement d'une image
def load_and_preprocess_image(path):
    # 1️⃣ Lecture de l'image à partir du chemin spécifié
    # 'cv2.IMREAD_GRAYSCALE' convertit l'image en niveaux de gris (1 canal)
    # Les radiographies (comme celles du dataset MIMIC-CXR) sont en noir et blanc,
    # donc cette option réduit la complexité sans perte d'information utile.
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    # 2️⃣ Redimensionnement de l'image à la taille standard définie (224x224)
    # Cela garantit une taille d'entrée uniforme pour le réseau de neurones
    img = cv2.resize(img, IMG_SIZE)

    # 3️⃣ Normalisation des valeurs de pixels entre 0 et 1
    # Les valeurs d'origine (0–255) sont divisées par 255.0
    # La normalisation accélère et stabilise l'entraînement du modèle
    img = img / 255.0  # normalisation 0-1

    # 4️⃣ Ajout d'une dimension supplémentaire pour le canal (grayscale → 1 canal)
    # La nouvelle forme devient (224, 224, 1), conforme aux attentes de TensorFlow/Keras
    img = np.expand_dims(img, axis=-1)

    # 5️⃣ Retourne l'image prétraitée
    return img



In [ ]:
# Importation du module 'os' :
# Permet d’interagir avec le système d’exploitation, notamment pour gérer les chemins de fichiers,
# créer des dossiers ou lister les fichiers dans un répertoire.
import os

# Importation de la bibliothèque OpenCV (cv2) :
# Utilisée pour le traitement d'images : lecture, redimensionnement, conversion de couleurs, filtrage, etc.
import cv2

# Importation de NumPy :
# Fournit des structures de données performantes pour manipuler des tableaux numériques (matrices, tenseurs),
# très utilisées pour représenter les images et effectuer des opérations mathématiques rapides.
import numpy as np

# Importation de Pandas :
# Permet de manipuler facilement les données tabulaires (comme les CSV) sous forme de DataFrames,
# pour organiser les métadonnées associées aux images (ex. chemins, labels, etc.)
import pandas as pd

# Importation de Matplotlib :
# Bibliothèque de visualisation, utilisée pour afficher des images, des courbes d'entraînement, etc.
import matplotlib.pyplot as plt

# Importation de tqdm :
# Ajoute des barres de progression dans les boucles (for, map...) pour suivre visuellement l’avancement du traitement
from tqdm import tqdm

# Importation des modules de Keras (intégré à TensorFlow) :
# 'layers' permet de créer les différentes couches du réseau de neurones (Conv2D, Dense, Dropout, etc.)
# 'models' permet de construire, compiler et entraîner le modèle de deep learning
from tensorflow.keras import layers, models


In [ ]:
# Définit le chemin de base vers le dossier du dataset MIMIC-CXR téléchargé via Kaggle
# Ce dossier contient les fichiers CSV et les sous-dossiers d’images.
base_path = "/root/.cache/kagglehub/datasets/simhadrisadaram/mimic-cxr-dataset/versions/2"

# Construit le chemin complet vers le dossier contenant les images officielles du dataset
# "official_data_iccv_final" est le sous-dossier où se trouvent les fichiers d'images radiographiques.
img_root = os.path.join(base_path, "official_data_iccv_final")

# Charge le fichier CSV d'entraînement du dataset MIMIC-CXR dans un DataFrame Pandas.
# Ce fichier contient généralement :
# - les chemins relatifs des images (ex: "files/patient12345/image1.png")
# - les métadonnées (vue, sexe, âge, etc.)
# - et les étiquettes associées aux maladies ou anomalies détectées.
df = pd.read_csv(os.path.join(base_path, "mimic_cxr_aug_train.csv"))



In [ ]:
# Importation du module 'ast' (Abstract Syntax Trees)
# Ce module permet de convertir en toute sécurité une chaîne représentant une structure Python
# (ex. "['PA', 'LATERAL']") en un objet Python réel (liste, dictionnaire, etc.)
import ast


# Définition d'une fonction utilitaire pour vérifier si une radiographie est une vue frontale
def has_frontal_view(views_str):
    try:
        # Convertit la chaîne de texte (ex: "['PA', 'LATERAL']") en liste Python
        views = ast.literal_eval(views_str)

        # Vérifie si la liste contient une vue frontale :
        # 'PA' (Posteroanterior) ou 'AP' (Anteroposterior)
        # Ces deux types de vues représentent les radiographies prises de face
        return any(v in ['PA', 'AP'] for v in views)

    except:
        # Si la conversion échoue (ex : valeur manquante ou format invalide),
        # la fonction renvoie False pour ignorer cette ligne
        return False


# Filtrage du DataFrame pour ne garder que les radiographies frontales
# On applique la fonction 'has_frontal_view' à la colonne 'view'
# et on crée une copie du sous-ensemble correspondant
df_frontal = df[df['view'].apply(has_frontal_view)].copy()


# Extraction du chemin relatif de l’image
# La colonne 'image' contient une chaîne représentant une liste (ex : "['files/patient123/image1.png']")
# On utilise 'ast.literal_eval' pour en extraire le premier élément (le vrai chemin de l'image)
df_frontal['image_path'] = df_frontal['image'].apply(
    lambda x: ast.literal_eval(x)[0] if isinstance(x, str) else None
)


# Construction du chemin absolu complet vers chaque image
# On combine le chemin racine 'img_root' avec le chemin relatif extrait ci-dessus
# On retire le préfixe "files/" pour éviter les doublons dans le chemin
df_frontal['full_path'] = df_frontal['image_path'].apply(
    lambda p: os.path.join(img_root, p.replace("files/", "")) if p else None
)


# Affiche le nombre total d’images frontales détectées
print(" Images frontales :", len(df_frontal))

# Affiche les premières lignes du DataFrame avec les identifiants patients et chemins d’accès complets
print(df_frontal[['subject_id', 'full_path']].head())


In [ ]:
# Définition d'une fonction qui construit un modèle U-Net pour la segmentation d'images
def unet_model(input_size=(224, 224, 1)):
    # Couche d'entrée — attend des images de taille 224x224 avec 1 canal (niveaux de gris)
    inputs = layers.Input(input_size)

    # ===============================
    #  ENCODER (Partie contractante)
    # ===============================

    # Bloc 1 — Première couche de convolution
    c1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    c1 = layers.Conv2D(64, 3, activation='relu', padding='same')(c1)
    p1 = layers.MaxPooling2D()(c1)  # Réduction de la taille (downsampling)

    # Bloc 2
    c2 = layers.Conv2D(128, 3, activation='relu', padding='same')(p1)
    c2 = layers.Conv2D(128, 3, activation='relu', padding='same')(c2)
    p2 = layers.MaxPooling2D()(c2)

    # Bloc 3
    c3 = layers.Conv2D(256, 3, activation='relu', padding='same')(p2)
    c3 = layers.Conv2D(256, 3, activation='relu', padding='same')(c3)
    p3 = layers.MaxPooling2D()(c3)

    # ===============================
    #  BOTTLENECK (partie centrale)
    # ===============================
    # Partie la plus profonde du réseau : capture les caractéristiques les plus abstraites
    c4 = layers.Conv2D(512, 3, activation='relu', padding='same')(p3)
    c4 = layers.Conv2D(512, 3, activation='relu', padding='same')(c4)

    # ===============================
    #  DECODER (Partie expansive)
    # ===============================

    # Bloc 4 — Upsampling + concaténation avec la couche correspondante du chemin descendant (skip connection)
    u5 = layers.UpSampling2D()(c4)
    u5 = layers.concatenate([u5, c3])
    c5 = layers.Conv2D(256, 3, activation='relu', padding='same')(u5)
    c5 = layers.Conv2D(256, 3, activation='relu', padding='same')(c5)

    # Bloc 5
    u6 = layers.UpSampling2D()(c5)
    u6 = layers.concatenate([u6, c2])
    c6 = layers.Conv2D(128, 3, activation='relu', padding='same')(u6)
    c6 = layers.Conv2D(128, 3, activation='relu', padding='same')(c6)

    # Bloc 6
    u7 = layers.UpSampling2D()(c6)
    u7 = layers.concatenate([u7, c1])
    c7 = layers.Conv2D(64, 3, activation='relu', padding='same')(u7)
    c7 = layers.Conv2D(64, 3, activation='relu', padding='same')(c7)

    # ===============================
    # Sortie finale
    # ===============================

    # Couche de sortie : 1 canal de sortie (masque binaire)
    # Activation sigmoïde → valeurs entre 0 et 1 pour la segmentation binaire
    outputs = layers.Conv2D(1, 1, activation='sigmoid')(c7)

    # Création du modèle complet
    model = models.Model(inputs, outputs)
    return model


# ===============================
#  Compilation et résumé du modèle
# ===============================

# Création de l’instance du modèle
unet = unet_model()

# Compilation du modèle :
# - Optimiseur Adam (efficace pour la segmentation)
# - Fonction de perte binaire (car masque 0/1)
unet.compile(optimizer='adam', loss='binary_crossentropy')

# Affiche la structure complète du modèle U-Net
unet.summary()


In [ ]:
# Définition de la taille standard des images
# Les radiographies seront redimensionnées à 224x224 pixels pour être compatibles avec le modèle U-Net
IMG_SIZE = (224, 224)


# Fonction de prétraitement et de segmentation d’une image
def preprocess_and_segment(image_path):
    # Lecture de l'image en niveaux de gris
    # cv2.IMREAD_GRAYSCALE permet de réduire les images à un seul canal (utile pour les radiographies)
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Vérifie que l'image a bien été chargée (évite les erreurs si le fichier est manquant ou corrompu)
    if img is None:
        return None

    #  Redimensionnement de l'image à la taille standard (224x224)
    img = cv2.resize(img, IMG_SIZE)

    #  Normalisation des valeurs de pixels entre 0 et 1 (améliore la stabilité du modèle)
    img_norm = img / 255.0

    #  Ajout de dimensions supplémentaires :
    # - La première (axis=0) correspond au batch (1 image)
    # - La dernière (axis=-1) correspond au canal (grayscale → 1)
    # Forme finale : (1, 224, 224, 1)
    inp = np.expand_dims(img_norm, axis=(0, -1))

    #  Prédiction du masque de segmentation à partir du modèle U-Net entraîné
    # 'verbose=0' évite les messages de progression à chaque prédiction
    mask = unet.predict(inp, verbose=0)[0, :, :, 0]

    #  Seuillage du masque : on garde les valeurs supérieures à 0.5 (probabilité > 50%)
    # Cela crée un masque binaire (0 ou 1)
    mask = (mask > 0.5).astype(np.uint8)

    #  Application du masque sur l'image originale
    # Les zones non segmentées (0) deviennent noires, les zones détectées sont conservées
    img_masked = cv2.bitwise_and(img, img, mask=mask)

    #  Normalisation de la sortie pour un affichage facile (valeurs 0–1)
    return img_masked / 255.0


# ===============================
#  Application sur un ensemble d'images
# ===============================

X_preprocessed = []  # liste pour stocker les images segmentées

# Récupération de la liste complète des chemins des images frontales
paths = df_frontal['full_path'].tolist()

# Boucle sur un échantillon des 200 premières images pour tester le pipeline
for path in tqdm(paths[:200]):  # tqdm affiche une barre de progression
    # Vérifie que le fichier existe
    if os.path.exists(path):
        # Prétraite et segmente l'image
        img = preprocess_and_segment(path)
        # Ajoute l'image segmentée si elle a bien été traitée
        if img is not None:
            X_preprocessed.append(img)

# Conversion de la liste en tableau NumPy (forme : nombre_images × 224 × 224)
X_preprocessed = np.array(X_preprocessed)

# Affiche la taille finale du jeu de données segmenté
print("Données prêtes :", X_preprocessed.shape)



In [ ]:
# Affiche le premier chemin complet d’image contenu dans le DataFrame df_frontal
# Cela permet de vérifier que les chemins ont bien été construits lors des étapes précédentes
print("Exemple de chemin :", df_frontal['full_path'].iloc[0])

# Vérifie si le fichier correspondant à ce chemin existe réellement sur le disque
# os.path.exists() renvoie True si le fichier est trouvé, sinon False
print("Existe :", os.path.exists(df_frontal['full_path'].iloc[0]))



In [ ]:
path = "/root/.cache/kagglehub/datasets/simhadrisadaram/mimic-cxr-dataset/versions/2/official_data_iccv_final/files/p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg"

In [ ]:
# Importation des bibliothèques nécessaires
# 'os' → permet de vérifier l'existence des fichiers et manipuler les chemins
# 'cv2' → utilisé pour lire et traiter les images (OpenCV)
# 'matplotlib.pyplot' → sert à afficher les images et graphiques
import os, cv2, matplotlib.pyplot as plt


# Vérifie si le fichier image existe à l'emplacement indiqué par la variable 'path'
# Cela évite les erreurs en cas de chemin incorrect ou de fichier manquant
print("Existe :", os.path.exists(path))


# Lecture de l'image en niveaux de gris (grayscale)
# cv2.IMREAD_GRAYSCALE charge l’image avec un seul canal (valeurs 0–255)
# Utile pour les radiographies, qui sont déjà monochromes
img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)


# Affichage de l'image avec Matplotlib
# cmap='gray' → indique à Matplotlib d’utiliser une échelle de gris
plt.imshow(img, cmap='gray')

# Supprime les axes (graduations, étiquettes) pour un affichage plus propre
plt.axis('off')

# Affiche effectivement l’image dans le notebook
plt.show()



In [ ]:
img_root = "/root/.cache/kagglehub/datasets/simhadrisadaram/mimic-cxr-dataset/versions/2/official_data_iccv_final"

df_frontal['full_path'] = df_frontal['image_path'].apply(
    lambda p: os.path.join(img_root, "files", p.split("files/")[-1]) if isinstance(p, str) else None
)

# Vérification
sample_path = df_frontal['full_path'].iloc[0]
print(" Exemple de chemin :", sample_path)
print("Existe :", os.path.exists(sample_path))


In [ ]:
# Importation des bibliothèques nécessaires
import matplotlib.pyplot as plt   # Pour afficher les images
import cv2                        # Pour lire les fichiers image (OpenCV)
import os                         # Pour vérifier l’existence des fichiers et manipuler les chemins


# 🔍 Sélection des chemins d'images valides
# On parcourt la colonne 'full_path' du DataFrame, en supprimant les valeurs manquantes (NaN)
# et on garde uniquement les chemins dont le fichier existe réellement sur le disque
valid_paths = [p for p in df_frontal['full_path'].dropna().tolist() if os.path.exists(p)]

# On sélectionne les deux premières images valides pour l’affichage
samples = valid_paths[:2]


# 🖼️ Création d’une figure Matplotlib pour afficher les images côte à côte
plt.figure(figsize=(8, 4))  # Taille totale de la figure (largeur x hauteur en pouces)

# Boucle sur les images sélectionnées
for i, path in enumerate(samples):
    # Lecture de l'image en niveaux de gris (1 canal)
    img = cv2.imread(path, cv2.IMREAD_GRAYSCALE)

    # Vérification : si l’image n’a pas pu être chargée, on passe à la suivante
    if img is None:
        print(f"⚠️ Impossible de lire : {path}")
        continue

    # Création d’un sous-graphique pour chaque image
    plt.subplot(1, 2, i + 1)  # 1 ligne, 2 colonnes, position i+1

    # Affichage de l’image avec une échelle de gris
    plt.imshow(img, cmap='gray')

    # Ajout d’un titre indiquant le numéro et le nom du fichier image
    plt.title(f"Image {i+1}\n{os.path.basename(path)}")

    # Suppression des axes pour un affichage plus propre
    plt.axis('off')


# Ajuste automatiquement la disposition pour éviter que les images ou titres se chevauchent
plt.tight_layout()

# Affiche les images dans le notebook
plt.show()

